In [24]:
import os
os.chdir("../")

In [25]:
%pwd

'c:\\Users\\farha\\projects\\AI-Health-Diagnosis-Assistant'

In [26]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [28]:
extracted_data=load_pdf_file(data='Data/')

In [20]:
extracted_data

NameError: name 'extracted_data' is not defined

In [11]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [19]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

NameError: name 'text_split' is not defined

In [18]:
text_chunks

NameError: name 'text_chunks' is not defined

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
try:
    from sentence_transformers import SentenceTransformer
    print("Import worked!")
except Exception as e:
    print("Import failed:", e)

c:\Users\farha\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import worked!


In [16]:
embeddings = download_hugging_face_embeddings()

C:\Users\farha\AppData\Local\Temp\ipykernel_14804\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
query_result

[-0.03447723761200905,
 0.031023213639855385,
 0.006734990980476141,
 0.02610895223915577,
 -0.03936200216412544,
 -0.16030248999595642,
 0.06692393124103546,
 -0.006441502831876278,
 -0.04745049029588699,
 0.014758865348994732,
 0.07087529450654984,
 0.05552753433585167,
 0.019193345680832863,
 -0.026251327246427536,
 -0.010109513066709042,
 -0.026940496638417244,
 0.022307435050606728,
 -0.022226642817258835,
 -0.1496925801038742,
 -0.01749304123222828,
 0.007676258217543364,
 0.05435232073068619,
 0.0032544711139053106,
 0.031725890934467316,
 -0.0846213549375534,
 -0.029405983164906502,
 0.05159558728337288,
 0.04812406003475189,
 -0.0033148040529340506,
 -0.05827920883893967,
 0.04196924716234207,
 0.022210638970136642,
 0.1281888484954834,
 -0.022338991984725,
 -0.011656233109533787,
 0.06292837113142014,
 -0.03287629410624504,
 -0.09122603386640549,
 -0.031175388023257256,
 0.052699580788612366,
 0.0470348484814167,
 -0.08420310914516449,
 -0.030056176707148552,
 -0.020744847133

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-knowledge-base-index"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [16]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [17]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

NameError: name 'text_chunks' is not defined

In [12]:
# Import PineconeVectorStore from LangChain integration
# This allows LangChain to interact with an existing Pinecone vector index

# Loading my existing Pinecone index and attach an embedding model to it
# This does NOT recreate or modify the index — it just connects to it so you can use it for searching

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

NameError: name 'index_name' is not defined

In [11]:
docsearch

NameError: name 'docsearch' is not defined

In [83]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [86]:
retrieved_docs = retriever.invoke("what Arbovirus?")

In [85]:
retrieved_docs

[Document(id='d409fd46-6cf5-48b7-bf6e-c2cf85061f6b', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 420.0, 'page_label': '391', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 4505.0}, page_content='APSGN see Acute poststreptococcal\nglomerulonephritis\nAPTT see Partial thromboplastin time\nArachnodactyly see Marfan syndrome\nArbovirus encephalitis\nDefinition\nEncephalitis is a serious inflammation of the\nbrain, Arbovirus encephalitis is caused by a virus\nfrom the Arbovirus group. The termarbovirus stands\nfor Arthro-pod-borne virus because these viruses are\npassed to humans by members of the phylum\nArthropoda (which includes insects and spiders).\nDescription'),
 Document(id='d4714361-51c2-4e5c-8454-4e9e201872f7', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 1338

In [7]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    model="llama3.2", temperature=0.7, max_tokens=500
)


In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Prompt
system_prompt = (
    "You are a knowledgeable medical assistant designed to answer user questions "
    "based on the retrieved context. Carefully read the provided context and respond "
    "with a clear, accurate, and concise answer. If the information needed is not in the "
    "context, respond with 'I'm not sure based on the available information.' "
    "Limit your answer to a maximum of three sentences."
    "\n\n"
    "{context}"
)

# Create a chat-style prompt template using system and user messages
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),   # System instructions (invisible to user)
        ("human", "{input}"),        # User's question
    ]
)


In [10]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

NameError: name 'retriever' is not defined

In [96]:
response = rag_chain.invoke({"input": "what Arbovirus encephalitis?"})
print(response["answer"])

APIConnectionError: Connection error.

In [101]:
import os
print(os.environ.get("OPENAI_API_KEY"))

In [92]:
from dotenv import load_dotenv
import os

load_dotenv()  # This loads the .env file
print(os.getenv("OPENAI_API_KEY"))  # Should now print the key


In [47]:
import pinecone
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")

# ✅ Replace with your actual Pinecone environment
pinecone.init(api_key=api_key, environment="us-east-1")  

print("✅ Pinecone initialized!")


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )

